In [185]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [158]:
data_crab = np.loadtxt("CrabDatasetforTrain.txt",dtype = "double")
print(data_crab)

[[15.  11.9 32.5 ...  1.   0.   1. ]
 [18.2 13.7 38.8 ...  1.   0.   0. ]
 [18.9 16.7 36.3 ...  0.   1.   0. ]
 ...
 [12.6 12.2 26.1 ...  0.   1.   1. ]
 [19.7 16.7 39.9 ...  0.   1.   0. ]
 [ 9.6  7.9 20.1 ...  1.   0.   1. ]]


In [169]:
# split data
data_c_label = data_crab[:,-1]
# print(data_c_label)
# print(data_c_train.shape)
data_c_features = data_crab[:,0:-1]
# print(data_c_features)

In [184]:
kf = KFold(n_splits=4) # Define the split - into k folds
for train_index, test_index in kf.split(data_c_features):
    print("\nTRAIN:\n", train_index,"\n", "\nTEST:", test_index)
    # split
    X_train, X_test = data_c_features[train_index], data_c_features[test_index]
    y_train, y_test = data_c_label[train_index], data_c_label[test_index]

    # since is two-class classification, we can get mean from log likelihhod estimation
    X_train_label = data_crab[train_index]
    X_test_label = data_crab[test_index]
    # this is  the prior of class 1
    
    p_prior_1 =  X_train_label[ X_train_label[:,-1]==1.0 ].size/ X_train_label.shape[0]
    # this is the prior of class 0
    p_prior_0 =  X_train_label[ X_train_label[:,-1]==0.0 ].size/ X_train_label.shape[0]
    
    # num of data points?
#     data_crab.shape[0]
    mean1 = np.mean( X_train_label[X_train_label[:,-1] == 1.0][:,0:-1],axis=0)
    mean0 = np.mean( X_train_label[X_train_label[:,-1] == 0.0][:,0:-1],axis=0)
    
    cov1 = np.cov( X_train_label[X_train_label[:,-1]==1.0][:,0:-1].T)
    for i in range(7):
        cov1[i][i]+=3e-8
    cov0 = np.cov( X_train_label[X_train_label[:,-1]==0.0][:,0:-1].T)
    for i in range(7):
        cov0[i][i]+=3.5e-8
    
    y_hat = []
    for i in range(X_test_label.shape[0]):
        p_L_1 = multivariate_normal.pdf(X_test_label[i,0:-1],mean = mean1,cov=cov1)
        p_L_0 = multivariate_normal.pdf(X_test_label[i,0:-1],mean = mean0,cov=cov0)
        
        # compute the posterior probability of each data point
        p_pos_1 = p_L_1*p_prior_1/(p_L_1*p_prior_1+p_L_0*p_prior_0)
        p_pos_0 = p_L_0*p_prior_0/(p_L_1*p_prior_1+p_L_0*p_prior_0)
        
        # decide
        if p_pos_0 > p_pos_1:
            y_hat.append(0.0)
        else:
            y_hat.append(1.0)
     # compute condusion matrix   
    confusion_mat = confusion_matrix(y_test,y_hat)
#     fpr, tpr, thresholds = metrics.roc_curve(y_test, y_hat)
    print("confusion_matrix is :\n",confusion_mat)
            


TRAIN:
 [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51
  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69
  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133] 
 
TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33]
confusion_matrix is :
 [[15  0]
 [ 0 19]]

TRAIN:
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  68  69
  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133] 
 
TEST: [34

In [165]:
cov1 = np.cov( X_train_label[X_train_label[:,-1]==1.0][:,0:-1].T)
cov1.shape
print("mean1 is \n",mean1,"\n")
print("covariance is :\n",cov1)
# print("inverse of covariance:",np.linalg.inv(cov1))
# print("determinant of cov",np.linalg.det(cov1))
print(np.linalg.det(cov1)+1e-16)
print((np.linalg.det(cov1)+1e-16)**0.5)
cov1.shape

mean1 is 
 [13.86122449 11.82040816 29.54489796 34.09795918 12.35306122  0.44897959
  0.55102041] 

covariance is :
 [[ 8.43534014  5.90018282 19.14323554 21.64700255  8.29230867  0.30943878
  -0.30943878]
 [ 5.90018282  5.03665816 13.40885629 15.32712585  5.97201956 -0.15935374
   0.15935374]
 [19.14323554 13.40885629 43.94919218 49.54217687 18.98673469  0.81067177
  -0.81067177]
 [21.64700255 15.32712585 49.54217687 56.05603741 21.47281888  0.82593537
  -0.82593537]
 [ 8.29230867  5.97201956 18.98673469 21.47281888  8.38087585  0.27359694
  -0.27359694]
 [ 0.30943878 -0.15935374  0.81067177  0.82593537  0.27359694  0.25255102
  -0.25255102]
 [-0.30943878  0.15935374 -0.81067177 -0.82593537 -0.27359694 -0.25255102
   0.25255102]]
9.96020929566324e-17
9.980084817106136e-09


(7, 7)

In [43]:
data_crab.shape[0]

134

In [117]:
X_test

array([[15. , 11.9, 32.5, 37.2, 13.6,  1. ,  0. ],
       [18.2, 13.7, 38.8, 42.7, 17.2,  1. ,  0. ],
       [18.9, 16.7, 36.3, 41.7, 15.3,  0. ,  1. ],
       [12.9, 11. , 26.8, 30.9, 11.4,  1. ,  0. ],
       [15.8, 15. , 34.5, 40.3, 15.3,  0. ,  1. ],
       [12.6, 10. , 27.7, 31.7, 11.4,  1. ,  0. ],
       [12.8, 10.9, 27.4, 31.5, 11. ,  1. ,  0. ],
       [19.4, 14.1, 39.1, 43.2, 17.8,  1. ,  0. ],
       [15.6, 13.9, 32.8, 37.9, 13.4,  0. ,  1. ],
       [12.6, 11.5, 25. , 28.1, 11.5,  0. ,  1. ],
       [12. , 10.7, 24.6, 28.9, 10.5,  0. ,  1. ],
       [19.8, 14.2, 43.2, 49.7, 18.6,  1. ,  0. ],
       [20.6, 14.4, 42.8, 46.5, 19.6,  1. ,  0. ],
       [ 8.8,  7.7, 18.1, 20.8,  7.4,  1. ,  0. ],
       [20.1, 17.2, 39.8, 44.1, 18.6,  0. ,  1. ],
       [20. , 16.7, 40.4, 45.1, 17.7,  0. ,  1. ],
       [11.4,  9.2, 21.7, 24.1,  9.7,  0. ,  1. ],
       [10.4,  9.7, 21.7, 25.4,  8.3,  0. ,  1. ],
       [17.7, 13.6, 38.7, 44.5, 16. ,  1. ,  0. ],
       [16.2, 15.2, 34.5, 40.1,

In [51]:
# mean1 of all features
mean1 = np.mean(data_crab[data_c_label == 1.0],axis=0)
print(mean1)

[13.88823529 11.86029412 29.61617647 34.18529412 12.36323529  0.47058824
  0.52941176  1.        ]


In [53]:
var1 = np.var(data_crab[data_c_label == 1.0],axis=0)
var2 = np.var(data_crab[data_c_label == 0.0],axis=0)
var1

array([ 8.40250865,  4.88739403, 43.93753244, 56.16507785,  8.50173659,
        0.24913495,  0.24913495,  0.        ])

In [59]:
np.cov(data_c_features)

array([[204.36142857, 238.305     , 226.29095238, ..., 167.64880952,
        241.78619048, 125.44809524],
       [238.305     , 278.40333333, 263.80666667, ..., 195.12333333,
        282.52166667, 146.18666667],
       [226.29095238, 263.80666667, 252.86619048, ..., 187.0047619 ,
        269.31190476, 139.09761905],
       ...,
       [167.64880952, 195.12333333, 187.0047619 , ..., 138.77571429,
        199.02095238, 103.01714286],
       [241.78619048, 282.52166667, 269.31190476, ..., 199.02095238,
        288.01238095, 148.40452381],
       [125.44809524, 146.18666667, 139.09761905, ..., 103.01714286,
        148.40452381,  77.07142857]])

In [75]:
cov1 = np.cov(data_crab[data_c_label==1.0][:,0:-1])
cov1.shape

(68, 68)

In [77]:
data_crab[data_c_label==1.0][:,0:-1].shape

(68, 7)

In [79]:
p_L_1 = multivariate_normal.pdf(data_c_features,mean = mean1,cov=cov1)

ValueError: Dimension mismatch: array 'cov' is of shape (68, 68), but 'mean' is a vector of length 8.

In [81]:
data_c_features.shape

(134, 7)

In [92]:
data_crab[train_index]

array([[23.1, 15.7, 47.6, 52.8, 21.6,  1. ,  0. ,  0. ],
       [13.1, 10.9, 28.3, 32.4, 11.2,  1. ,  0. ,  1. ],
       [13.2, 11. , 27.1, 30.4, 12.2,  1. ,  0. ,  0. ],
       [11.8,  9.6, 24.2, 27.8,  9.7,  1. ,  0. ,  1. ],
       [11.6, 11. , 24.6, 28.5, 10.4,  0. ,  1. ,  1. ],
       [19.3, 13.5, 41.6, 47.4, 17.8,  1. ,  0. ,  1. ],
       [23.1, 20.2, 46.2, 52.5, 21.1,  0. ,  1. ,  0. ],
       [14.2, 10.6, 28.7, 31.7, 12.9,  1. ,  0. ,  0. ],
       [13.7, 11. , 27.5, 30.5, 12.2,  1. ,  0. ,  0. ],
       [16.4, 13. , 35.7, 41.8, 15.2,  1. ,  0. ,  1. ],
       [15. , 13.8, 31.7, 36.9, 14. ,  0. ,  1. ,  1. ],
       [16.8, 12.8, 36.2, 41.8, 14.9,  1. ,  0. ,  1. ],
       [ 8.1,  6.7, 16.1, 19. ,  7. ,  1. ,  0. ,  1. ],
       [14.2, 11.3, 29.2, 32.2, 13.5,  1. ,  0. ,  0. ],
       [14.1, 10.7, 28.7, 31.9, 13.3,  1. ,  0. ,  0. ],
       [17.4, 16.9, 38.2, 44.1, 16.6,  0. ,  1. ,  1. ],
       [15.2, 14.3, 33.9, 38.5, 14.7,  0. ,  1. ,  1. ],
       [21.4, 18. , 41.2, 46.2,